In [ ]:
import control.matlab as cnt
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from tf2ss import lsim

from sippy_unipi import functionset as fset
from sippy_unipi import system_identification

rcParams["figure.figsize"] = (9.0, 5.0)

## Define sampling time and Time vector

In [ ]:
sampling_time = 1.0  # [s]
end_time = 400  # [s]
npts = int(end_time / sampling_time) + 1
Time = np.linspace(0, end_time, npts)

## Define pseudo random binary sequence as input signal and white noise as noise signal

In [ ]:
# make input signal
switch_probability = 0.08  # [0..1]
Usim, _, _ = fset.GBN_seq(npts, switch_probability)

# make noise for input signal
white_noise_variance = [0.005]
e_t = fset.white_noise_var(Usim.size, white_noise_variance)[0]

## Define the system

### Numerator of noise transfer function has only one root: nc = 1

In [ ]:
NUM_H = [
    1.0,
    0.3,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
]

### Common denominator between input and noise transfer functions has 4 roots: na = 4

In [ ]:
DEN = [
    1.0,
    -2.21,
    1.7494,
    -0.584256,
    0.0684029,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
]

### Numerator of input transfer function has 3 roots: nb = 3

In [ ]:
NUM = [1.0, -2.07, 1.3146]

### Define transfer functions

In [ ]:
g_sample = cnt.tf(NUM, DEN, sampling_time)
h_sample = cnt.tf(NUM_H, DEN, sampling_time)
print("g()")
display(g_sample)
print("h()")
display(h_sample)

## Time responses

### Input reponse

In [ ]:
Y1, Time, Xsim = lsim(g_sample, Usim, Time)
plt.figure()
plt.plot(Time, Usim, label="u(t)")
plt.plot(Time, Y1, label="y(t)")
plt.xlabel("Time")
plt.title("Time response y(t)=g*u(t)")
plt.legend()
plt.grid()

### Noise response

In [ ]:
Y2, Time, Xsim = lsim(h_sample, e_t, Time)
plt.figure()
plt.plot(Time, e_t, label="e(t)")
plt.plot(Time, Y2, label="y(t)")
plt.xlabel("Time")
plt.title("Time response y(t)=h*e(t)")
plt.legend()
plt.grid()

## Compute total output 
$$Y_t = Y_1 + Y_2 = G.u + H.e$$

In [ ]:
Ytot = Y1 + Y2
Utot = Usim + e_t
plt.figure()
plt.plot(Time, Ytot, label="y_t(t)")
plt.plot(Time, Utot, label="u(t) + e(t)")
plt.xlabel("Time")
plt.title("Time response y_t(t)=g*u(t) + h*e(t)")
plt.legend()
plt.grid()

## Perform system identification from collected data

In [ ]:
Id_sys = system_identification(
    Ytot,
    Usim,
    "ARMAX",
    IC="BIC",
    na_ord=[2, 5],
    nb_ord=[1, 5],
    nc_ord=[0, 2],
    delays=[10, 13],
    max_iterations=1000,
)

## Check that output of the identified system is consistent

In [ ]:
Y_id1, Time, Xsim = lsim(Id_sys.G, Usim, Time)
Y_hid1, Time, Xsim = lsim(Id_sys.H, e_t, Time)
Y_idTot = Y_id1 + Y_hid1

In [ ]:
plt.figure(0)
plt.plot(Time, Usim)
plt.ylabel("Input GBN")
plt.xlabel("Time")
plt.title("Input, validation data (Switch probability=0.08)")
plt.grid()

In [ ]:
plt.figure(1)
plt.plot(Time, Ytot, label="Original system")
plt.plot(Time, Y_idTot, label="Identified system")
plt.grid()
plt.xlabel("Time")
plt.ylabel("y_tot")
plt.title("Gu+He (identification data)")
plt.legend()

In [ ]:
plt.figure(2)
plt.plot(Time, Y1, label="Original system")
plt.plot(Time, Y_id1, label="Identified system")
plt.ylabel("y_out")
plt.grid()
plt.xlabel("Time")
plt.legend()
plt.title("Gu (identification data)")

In [ ]:
plt.figure(3)
plt.plot(Time, Y2, label="Original system")
plt.plot(Time, Y_hid1, label="Identified system")
plt.ylabel("y_err")
plt.grid()
plt.xlabel("Time")
plt.legend()
plt.title("He (identification data)")

# Validation of the identified system: 
## Generate new time series for input and noise

In [ ]:
switch_probability = 0.07  # [0..1]
input_range = [0.5, 1.5]
U_valid, _, _ = fset.GBN_seq(npts, switch_probability, Range=input_range)
white_noise_variance = [0.01]
e_valid = fset.white_noise_var(U_valid.size, white_noise_variance)[0]

## Compute time responses for true system with new inputs

In [ ]:
Yvalid1, Time, Xsim = lsim(g_sample, U_valid, Time)
Yvalid2, Time, Xsim = lsim(h_sample, e_valid, Time)
Ytotvalid = Yvalid1 + Yvalid2

## Compute time responses for identified system with new inputs

In [ ]:
Yidvalid1, Time, Xsim = lsim(Id_sys.G, U_valid, Time)
Yidvalid2, Time, Xsim = lsim(Id_sys.H, e_valid, Time)
Yidtotvalid = Yidvalid1 + Yidvalid2

## Check responses are almost equal

In [ ]:
plt.figure(4)
plt.plot(Time, U_valid)
plt.ylabel("Input GBN")
plt.xlabel("Time")
plt.title("Input, validation data (Switch probability=0.07)")
plt.grid()

In [ ]:
plt.figure(5)
plt.plot(Time, Ytotvalid)
plt.plot(Time, Yidtotvalid)
plt.xlabel("Time")
plt.ylabel("y_tot")
plt.legend(["Original system", "Identified system"])
plt.grid()

rmse = np.round(np.sqrt(np.mean((Ytotvalid - Yidtotvalid) ** 2)), 2)
plt.title(f"Validation: Gu+He | RMSE = {rmse}")

In [ ]:
plt.figure(6)
plt.plot(Time, Yvalid1, label="Original system")
plt.plot(Time, Yidvalid1, label="Identified system")
plt.grid()
plt.xlabel("Time")
plt.ylabel("y_out")
plt.title("Gu (Validation)")
plt.legend()

In [ ]:
plt.figure(7)
plt.plot(Time, Yvalid2, label="Original system")
plt.plot(Time, Yidvalid2, label="Identified system")
plt.grid()
plt.xlabel("Time")
plt.ylabel("y_err")
plt.title("He (Validation)")
plt.legend()

In [ ]:
print("Original h()")
display(h_sample)
print("Identified h()")
display(Id_sys.H)

plt.figure()
_ = cnt.frequency_response([h_sample, Id_sys.H])

In [ ]:
print("Original g()")
display(g_sample)
print("Identified g()")
display(Id_sys.G)

plt.figure()
_ = cnt.frequency_response([g_sample, Id_sys.G])